# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [ ]:
import pandas as pd
import requests

# 1) Carregar a base de dados
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# Exibir as primeiras linhas e informações gerais para verificação
print("Informações iniciais do DataFrame:")
print(sinasc.head())
print("\nInformações sobre as colunas e tipos de dados:")
print(sinasc.info())
print("\nValores nulos por coluna:")
print(sinasc.isnull().sum())



In [ ]:
# 2) Selecione as colunas 'IDADEMAE', 'CONSULTAS' e 'DTNASC'.
#    Remova valores duplicados e valores nulos dessas colunas.
#    Crie uma nova coluna chamada 'DTNASC_JAN' que contenha a informação se a criança nasceu em Janeiro (1) ou não (0).

# Selecionando as colunas
df_filtered = sinasc[['IDADEMAE', 'CONSULTAS', 'DTNASC']].copy()

# Removendo valores duplicados
df_filtered.drop_duplicates(inplace=True)

# Removendo valores nulos
df_filtered.dropna(inplace=True)

# Convertendo 'DTNASC' para datetime para facilitar a extração do mês
df_filtered['DTNASC'] = pd.to_datetime(df_filtered['DTNASC'])

# Criando a coluna 'DTNASC_JAN'
df_filtered['DTNASC_JAN'] = (df_filtered['DTNASC'].dt.month == 1).astype(int)

print("\n2) DataFrame após seleção, remoção de duplicados/nulos e criação de 'DTNASC_JAN':")
print(df_filtered.head())
print("\nValores de 'DTNASC_JAN':")
print(df_filtered['DTNASC_JAN'].value_counts())


In [ ]:
# 3) Gerar gráficos que descrevam a distribuição de 'IDADEMAE' e 'CONSULTAS'.
#    Utilize os dados limpos do passo anterior.

plt.rc('figure', figsize=(10, 8))
fig, axes = plt.subplots(1, 2)

# Histograma para IDADEMAE
sns.histplot(ax=axes[0], data=df_filtered, x='IDADEMAE', bins=20, kde=True)
axes[0].set_title('Distribuição da Idade da Mãe')
axes[0].set_xlabel('Idade da Mãe')
axes[0].set_ylabel('Frequência')

# Contagem de barras para CONSULTAS (transformando em string para garantir que todas as categorias apareçam)
# De acordo com o PDF, CONSULTAS é C(01) e tem os valores:
# 1: Nenhuma
# 2: de 1 a 3
# 3: de 4 a 6
# 4: 7 e mais
# 9: Ignorado
# É melhor tratar como categórica.
consultas_map = {1: 'Nenhuma', 2: '1 a 3', 3: '4 a 6', 4: '7 e mais', 9: 'Ignorado'}
df_filtered['CONSULTAS_CAT'] = df_filtered['CONSULTAS'].map(consultas_map)
sns.countplot(ax=axes[1], data=df_filtered, x='CONSULTAS_CAT', order=consultas_map.values())
axes[1].set_title('Número de Consultas de Pré-natal')
axes[1].set_xlabel('Número de Consultas')
axes[1].set_ylabel('Contagem')

plt.tight_layout()
plt.show()

In [ ]:
# 4) Calcule a média e a mediana da idade da mãe.
#    Calcule a média e a mediana do número de consultas.
#    Utilize os dados limpos do passo 2.

media_idade_mae = df_filtered['IDADEMAE'].mean()
mediana_idade_mae = df_filtered['IDADEMAE'].median()

# Para consultas, como é uma variável categórica ordinal, a média e mediana não são tão informativas.
# Se quisermos uma "média" ou "mediana" que faça sentido, precisaríamos de uma representação numérica.
# Pelo dicionário de dados, 'CONSULTAS' é um campo C(01) com valores como 1, 2, 3, 4, 9.
# Se considerarmos estes como valores numéricos de categoria, podemos calcular.
# No entanto, é mais comum para variáveis categóricas ordinais usar a moda ou a distribuição.
# Se for para usar os valores brutos (1,2,3,4,9):
media_consultas_raw = df_filtered['CONSULTAS'].mean()
mediana_consultas_raw = df_filtered['CONSULTAS'].median()

print(f"\n4) Média da Idade da Mãe: {media_idade_mae:.2f}")
print(f"Mediana da Idade da Mãe: {mediana_idade_mae:.2f}")
print(f"Média do número de consultas (valores brutos 1,2,3,4,9): {media_consultas_raw:.2f}")
print(f"Mediana do número de consultas (valores brutos 1,2,3,4,9): {mediana_consultas_raw:.2f}")

In [ ]:
# 5) sinasc['DTNASC'] = pd.to_datetime(sinasc['DTNASC'])
sinasc['MES_NASCIMENTO'] = sinasc['DTNASC'].dt.month

# Mapear números dos meses para nomes para melhor visualização
meses_nomes = {
    1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril',
    5: 'Maio', 6: 'Junho', 7: 'Julho', 8: 'Agosto',
    9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
}
sinasc['MES_NASCIMENTO_NOME'] = sinasc['MES_NASCIMENTO'].map(meses_nomes)

plt.figure(figsize=(12, 6))
sns.countplot(data=sinasc, x='MES_NASCIMENTO_NOME', order=list(meses_nomes.values()))
plt.title('Frequência de Nascimentos por Mês (2019 - Rondônia)')
plt.xlabel('Mês de Nascimento')
plt.ylabel('Número de Nascimentos')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# 6) Criar um gráfico de dispersão com a idade da mãe no eixo x e o peso do bebê (APGAR5) no eixo y.
#    Considere apenas os registros em que 'APGAR5' não seja nulo.

# A coluna APGAR5 é o escore de Apgar no 5º minuto, que representa o peso do bebê.
# Pelo PDF: APGAR5 é um C(01) de 0 a 10. Ele representa a condição de saúde do recém-nascido.
# Não é o peso do bebê.
# Se o objetivo era peso, a coluna correta seria 'PESO'.
# Vamos considerar que a intenção era APGAR5, conforme o texto do exercício.

# Limpar valores nulos em IDADEMAE e APGAR5
df_apgar = sinasc.dropna(subset=['IDADEMAE', 'APGAR5']).copy()

# Converter APGAR5 para numérico. Ele pode ter sido lido como string.
# O PDF indica C(01), mas em arquivos CSV pode vir como int ou float.
df_apgar['APGAR5'] = pd.to_numeric(df_apgar['APGAR5'], errors='coerce')
df_apgar.dropna(subset=['APGAR5'], inplace=True) # Remover nulos que surgiram da conversão

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_apgar, x='IDADEMAE', y='APGAR5', alpha=0.5)
plt.title('Dispersão da Idade da Mãe vs. APGAR5')
plt.xlabel('Idade da Mãe')
plt.ylabel('APGAR5')
plt.grid(True)
plt.show()

In [ ]:
# 7) # Remover nulos de ESTCIVMAE e IDADEMAE
df_estciv = sinasc.dropna(subset=['ESTCIVMAE', 'IDADEMAE']).copy()

# Remover estado civil '9' (Ignorado)
df_estciv = df_estciv[df_estciv['ESTCIVMAE'] != 9]

# Mapear ESTCIVMAE para nomes significativos
# 1: Solteira
# 2: Casada
# 3: Viúva
# 4: Divorciada
# 5: União estável
estciv_map = {1: 'Solteira', 2: 'Casada', 3: 'Viúva', 4: 'Divorciada', 5: 'União Estável'}
df_estciv['ESTCIVMAE_NOME'] = df_estciv['ESTCIVMAE'].map(estciv_map)

plt.figure(figsize=(10, 6))
sns.boxplot(data=df_estciv, x='ESTCIVMAE_NOME', y='IDADEMAE')
plt.title('Boxplot da Idade da Mãe por Estado Civil')
plt.xlabel('Estado Civil da Mãe')
plt.ylabel('Idade da Mãe')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
# 8) # Remover nulos de ESCMAE
df_escmae = sinasc.dropna(subset=['ESCMAE']).copy()

# Remover escolaridade '9' (Ignorado)
df_escmae = df_escmae[df_escmae['ESCMAE'] != '9']

# Mapear ESCMAE para nomes significativos
# 1: Nenhuma
# 2: 1 a 3 anos
# 3: 4 a 7 anos
# 4: 8 a 11 anos
# 5: 12 anos e mais
escmae_map = {'1': 'Nenhuma', '2': '1 a 3 anos', '3': '4 a 7 anos', '4': '8 a 11 anos', '5': '12 anos e mais'}
df_escmae['ESCMAE_NOME'] = df_escmae['ESCMAE'].map(escmae_map)

# Definir a ordem das barras para que façam sentido
order_escmae = ['Nenhuma', '1 a 3 anos', '4 a 7 anos', '8 a 11 anos', '12 anos e mais']

plt.figure(figsize=(10, 6))
sns.countplot(data=df_escmae, x='ESCMAE_NOME', order=order_escmae)
plt.title('Frequência de Nascimentos por Escolaridade da Mãe')
plt.xlabel('Escolaridade da Mãe')
plt.ylabel('Número de Nascimentos')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [9]:
# 9) # Renomear as variáveis para que fiquem no snake_case

import pandas as pd # <-- Esta linha é crucial para definir 'pd'

# Carregar a base de dados (garantindo que estamos com a original para renomear)
try:
    sinasc = pd.read_csv('SINASC_RO_2019.csv')
except FileNotFoundError:
    print("Erro: O arquivo 'SINASC_RO_2019.csv' não foi encontrado. Certifique-se de que ele está no mesmo diretório.")
    # Adicione aqui um tratamento de erro ou uma forma de o usuário informar o caminho correto
    exit() # Interrompe a execução se o arquivo não for encontrado

# Função para converter nomes de colunas para snake_case
def to_snake_case(name):
    # Substitui espaços por underscores
    s1 = name.replace(' ', '_')
    # Insere underscore antes de letras maiúsculas (exceto a primeira) e converte para minúsculas
    # Ex: 'IDADEMAE' -> 'idade_mae', 'DTNASC' -> 'dt_nasc'
    return ''.join(['_' + c.lower() if c.isupper() and i > 0 else c for i, c in enumerate(s1)]).lstrip('_')

# Criar um dicionário de mapeamento dos nomes antigos para os novos nomes (snake_case)
new_columns = {col: to_snake_case(col) for col in sinasc.columns}

# Renomear as colunas do DataFrame usando inplace=True para modificar o DataFrame diretamente
sinasc.rename(columns=new_columns, inplace=True)

print("Nomes das colunas após renomeação para snake_case:")
print(sinasc.columns.tolist()) # .tolist() para uma visualização mais limpa na saída

Nomes das colunas após renomeação para snake_case:
['O_r_i_g_e_m', 'C_o_d_e_s_t_a_b', 'C_o_d_m_u_n_n_a_s_c', 'L_o_c_n_a_s_c', 'I_d_a_d_e_m_a_e', 'E_s_t_c_i_v_m_a_e', 'E_s_c_m_a_e', 'C_o_d_o_c_u_p_m_a_e', 'Q_t_d_f_i_l_v_i_v_o', 'Q_t_d_f_i_l_m_o_r_t', 'C_o_d_m_u_n_r_e_s', 'G_e_s_t_a_c_a_o', 'G_r_a_v_i_d_e_z', 'P_a_r_t_o', 'C_o_n_s_u_l_t_a_s', 'D_t_n_a_s_c', 'H_o_r_a_n_a_s_c', 'S_e_x_o', 'A_p_g_a_r1', 'A_p_g_a_r5', 'R_a_c_a_c_o_r', 'P_e_s_o', 'I_d_a_n_o_m_a_l', 'D_t_c_a_d_a_s_t_r_o', 'C_o_d_a_n_o_m_a_l', 'N_u_m_e_r_o_l_o_t_e', 'V_e_r_s_a_o_s_i_s_t', 'D_t_r_e_c_e_b_i_m', 'D_i_f_d_a_t_a', 'D_t_r_e_c_o_r_i_g_a', 'N_a_t_u_r_a_l_m_a_e', 'C_o_d_m_u_n_n_a_t_u', 'C_o_d_u_f_n_a_t_u', 'E_s_c_m_a_e2010', 'S_e_r_i_e_s_c_m_a_e', 'D_t_n_a_s_c_m_a_e', 'R_a_c_a_c_o_r_m_a_e', 'Q_t_d_g_e_s_t_a_n_t', 'Q_t_d_p_a_r_t_n_o_r', 'Q_t_d_p_a_r_t_c_e_s', 'I_d_a_d_e_p_a_i', 'D_t_u_l_t_m_e_n_s_t', 'S_e_m_a_g_e_s_t_a_c', 'T_p_m_e_t_e_s_t_i_m', 'C_o_n_s_p_r_e_n_a_t', 'M_e_s_p_r_e_n_a_t', 'T_p_a_p_r_e_s_e_n_t', 'S_t_t_r_